# Analýza a vizualizace dat

- Analýza a vizualizace dat 
    - Nástroje pro platformu Python
    - Pandas
    - Matplotlib
    - Další nástroje (numpy, scikit, ...)

## Úvodní cvičení

### Příprava dat (pro Excel)

> **Zadání**
> 
> Vytvořte homogenní strukturu, na základě které vytvoříte kontingenční tabulku (např. v Excelu) počítající sumu hodin
> - po učebních skupinách v kombinaci s typem lekce
> - po vyučujících v kombinaci s typem lekce
> - ...


Zadání napovídá, že bude potřeba homogenní struktura (tabulka). Je nutné vědět, co to je kontingenční tabulka (pivot table).

> **Povinné video**
>
> [Základy kontingenční tabulky | EduTV 10 min](https://www.youtube.com/watch?v=IWse9RPLDgM)

Zadání současně říká, jaká data musí být přítomna.
- učební skupina
- typ lekce
- délka lekce
- vyučující
- ...
- předmět

Výchozími daty je json soubor, se kterým jsme již dříve pracovali (aplikace rozvrh hodin).


> **Mikroúkol**
>
> Napište funkci pro spojení textů v listu s možností určení oddělovače

In [ ]:
def joinStr(delimiter, strList):
    result = ''
    
    #
    #
    #
    
    return result

> **Řešení**

In [11]:
def joinStr(delimiter, strList):
    result = ''
    for index, item in enumerate(strList):
        if index == 0:
            result = item
        else:
            result = result + delimiter + item
    return result

print(joinStr(', ', ['A', 'B', 'C']))

A, B, C


In [2]:
import json

with open('rozvrh/data.json', encoding='utf-8') as inputFile:
    data = json.load(inputFile)

In [8]:
for index, item in enumerate(data['events']):
    print(item)
    if index > 2:
        break

{'id': '2D2B1C90-F4FF-11EB-9B74-520D00000000', 'typeId': 'F00CB650-9768-11EB-87D2-030800000000', 'startTime': {'hours': 14, 'minutes': 30}, 'endTime': {'hours': 16, 'minutes': 0}, 'dateCode': '2021-11-15', 'date': {'day': 15, 'month': 11, 'year': 2021}, 'categoryId': 1, 'subjectId': 32269, 'subjectName': 'Letecké elektronické systémy II', 'departmentId': 441, 'departmentName': 'FVT-K206', 'topic': '3. Syntéza kmitočtu ', 'topicId': 62645, 'masterId': '62645/2', 'timeslotsId': 1, 'timeslotsName': 'BR', 'lessonOrder': 8, 'lessonFormId': 2, 'lessonFormName': 'CV', 'lessonUnit': 45, 'lessonsCount': 2, 'groupsIds': ['A8CFD4D0-8CA4-11EB-BA48-520D00000000'], 'groupsNames': ['23-3LT-AV-C'], 'groupsEntryYearsIds': [], 'classroomsIds': [331], 'classroomsNames': ['Č1/120'], 'classroomsAreasIds': [2], 'teachersIds': [540], 'teachersNames': ['Němeček, Jiří'], 'teachersDepartmentsIds': [441], 'isLocked': True}
{'id': 'C65A0170-F4FE-11EB-9B74-520D00000000', 'typeId': 'EFAE42A0-9768-11EB-87D2-03080000

Na základě výpisu připravíme mapovací funkci

> **Připomenutí**
> 
> Využití generátorů

In [12]:
def updateData(data):
    for item in data:
        yield item + 1
    
inputData = [1, 2, 3, 4]
resultData = list(updateData(inputData))
print(resultData)

[2, 3, 4, 5]


Implementace funkce

In [27]:
def dataExtractor(events):
    for item in events:
        resultItem = {
            'id': item['id'],
            'subjectName': item['subjectName'] if 'subjectName' in item else 'unknown',
            'departmentName': item['departmentName'] if 'departmentName' in item else 'unknown',
            'topic': item['topic'] if 'topic' in item else 'unknown',
            'lessonFormName': item['lessonFormName'] if 'lessonFormName' in item else 'unknown',
            'lessonsCount': item['lessonsCount'] if 'lessonsCount' in item else 0,
            'groups': joinStr(', ', item['groupsNames']) if 'groupsNames' in item else 'unknown',
        }
        for teacher in item['teachersNames']:
            newResult = {
                **resultItem, 
                'teacher': teacher
            }
            yield newResult
    pass

In [28]:
import json

with open('rozvrh/data.json', encoding='utf-8') as inputFile:
    data = json.load(inputFile)
    
dataForPivotTable = list(dataExtractor(data['events']))
for index, item in enumerate(dataForPivotTable):
    print(item)
    if index > 2:
        break
        

{'id': '2D2B1C90-F4FF-11EB-9B74-520D00000000', 'subjectName': 'Letecké elektronické systémy II', 'departmentName': 'FVT-K206', 'topic': '3. Syntéza kmitočtu ', 'lessonFormName': 'CV', 'lessonsCount': 2, 'groups': '23-3LT-AV-C', 'teacher': 'Němeček, Jiří'}
{'id': 'C65A0170-F4FE-11EB-9B74-520D00000000', 'subjectName': 'Letecké elektronické systémy II', 'departmentName': 'FVT-K206', 'topic': '1. Letecké rádiové přijímače', 'lessonFormName': 'LAB', 'lessonsCount': 2, 'groups': '23-3LT-AV-C', 'teacher': 'Němeček, Jiří'}
{'id': '396CC6C0-F4FF-11EB-9B74-520D00000000', 'subjectName': 'Letecké elektronické systémy II', 'departmentName': 'FVT-K206', 'topic': '3. Syntéza kmitočtu ', 'lessonFormName': 'LAB', 'lessonsCount': 2, 'groups': '23-3LT-AV-C', 'teacher': 'Němeček, Jiří'}
{'id': '7B7F24E0-F4FF-11EB-9B74-520D00000000', 'subjectName': 'Letecké elektronické systémy II', 'departmentName': 'FVT-K206', 'topic': '5. Úhloměrné navigační systémy NDB, VOR, DVOR, TACAN, VORTAC ', 'lessonFormName': 'CV

## Pandas

Pandas je jedna z největších knihoven pro Python. Jedná se o opensource! 
Dokumentaci lze nalézt [zde](https://pandas.pydata.org/).

> **Doporučené video**
>
> [Complete Python Pandas Data Science Tutorial! 1h](https://www.youtube.com/watch?v=vmEHCJofslg)


### DataFrame

Jedním ze základních prvků je [DataFrame](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html).

Zobrazíme si získaná data v tabulkové podobě.

In [29]:
import json
import pandas as pd

with open('rozvrh/data.json', encoding='utf-8') as inputFile:
    data = json.load(inputFile)
    
df = pd.DataFrame(dataExtractor(data['events']))
df

,id,subjectName,departmentName,topic,lessonFormName,lessonsCount,groups,teacher
0,2D2B1C90-F4FF-11EB-9B74-520D00000000,Letecké elektronické systémy II,FVT-K206,3. Syntéza kmitočtu,CV,2,23-3LT-AV-C,"Němeček, Jiří"
1,C65A0170-F4FE-11EB-9B74-520D00000000,Letecké elektronické systémy II,FVT-K206,1. Letecké rádiové přijímače,LAB,2,23-3LT-AV-C,"Němeček, Jiří"
2,396CC6C0-F4FF-11EB-9B74-520D00000000,Letecké elektronické systémy II,FVT-K206,3. Syntéza kmitočtu,LAB,2,23-3LT-AV-C,"Němeček, Jiří"
3,7B7F24E0-F4FF-11EB-9B74-520D00000000,Letecké elektronické systémy II,FVT-K206,"5. Úhloměrné navigační systémy NDB, VOR, DVOR,...",CV,2,23-3LT-AV-C,"Andrle, Miloš"
4,04E49860-F4FF-11EB-9B74-520D00000000,Letecké elektronické systémy II,FVT-K206,2. Letecké rádiové vysílače,LAB,2,23-3LT-AV-C,"Němeček, Jiří"
...,...,...,...,...,...,...,...,...
69258,0750ACE9-323D-4E14-B3CC-ABCDF66F96C2,unknown,FVL-K105,Válečné studie,ZK,0,"5D, 5DVZS, 5CH, 5MIZ, 5MT, 5PZ, 5ŘFZ, 5ŘLZ, 5V...","Staněk, Jaroslav"
69259,DFB9C331-9C9C-4485-B06F-ACCB19E4757E,unknown,FVL-K102,Management,ZK,0,"12-5ŘPOS1, 12-5ŘPOS2, 12-5ŘPOS3, 12-5ŘPOS4, 12...","Richter, Jiří"
69260,3292D722-AC2B-49B8-8E48-5B4EFF2084A2,unknown,FVT-K217,Základy elektrotechniky,ZK,0,"22-3GMOB-C, 22-3LT-C, 22-3TOMO-C, 22-3ZM-C LAB...","Kohl, Zdeněk"
69261,89D2BBB0-F582-4B6C-B8DB-D1D7D0F789CA,unknown,FVT-K217,Základy elektrotechniky,ZK,0,"22-3GMOB-C, 22-3LT-C, 22-3TOMO-C, 22-3ZM-C LAB...","Kohl, Zdeněk"


In [36]:
import numpy as np
pd.pivot_table(df, values='lessonsCount', index=['departmentName', 'teacher'], columns=['lessonFormName'], aggfunc=np.sum, fill_value='')

lessonFormName                       CV EX    J KOL    KON KRZ   LAB     P  \
departmentName teacher                                                       
CBVSS          Bauer, Miloslav                                         2.0   
               Baxa, Fabian        38.0              734.0            30.0   
               Botik, Martin                                           2.0   
               Botík, Martin                          12.0                   
               Brizgalová, Lenka            2.0                              
...                                 ... ..  ...  ..    ...  ..   ...   ...   
ÚOPZHN         Trefilík, Dušan                                        28.0   
               Zahradníček, Radim                                     32.0   
               Švec, Libor                                                   
               Žeravík, Jiří                                          54.0   
               Žuja, Petr          10.0                         54.0  44.0   

lessonFormName                        PV PX    SEM   SMP STŽ  SZK   TER   ZK  \
departmentName teacher                                                         
CBVSS          Bauer, Miloslav                                                 
               Baxa, Fabian                  149.0   8.0                 0.0   
               Botik, Martin                                                   
               Botík, Martin                 348.0  12.0           36.0        
               Brizgalová, Lenka                                               
...                                  ... ..    ...   ...  ..  ...   ...  ...   
ÚOPZHN         Trefilík, Dušan                 4.0                             
               Zahradníček, Radim              2.0            0.0        0.0   
               Švec, Libor                                    0.0              
               Žeravík, Jiří                                                   
               Žuja, Petr          325.0                                       

lessonFormName                    unknown  
departmentName teacher                     
CBVSS          Bauer, Miloslav             
               Baxa, Fabian                
               Botik, Martin               
               Botík, Martin               
               Brizgalová, Lenka           
...                                   ...  
ÚOPZHN         Trefilík, Dušan             
               Zahradníček, Radim          
               Švec, Libor                 
               Žeravík, Jiří               
               Žuja, Petr                  

[818 rows x 17 columns]

### Selekce

### Projekce

### Pivot Table

## Matplotlib

### Intro

### Liniové grafy

### Datové grafy

## Další nástroje